<a href="https://colab.research.google.com/github/TahseenSust/ML-assignment/blob/main/labFinal/MLLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/Colab Notebooks/datasetofAI/CNN_dataset.zip", "/content/CNN_dataset")

In [2]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define the path to the dataset
data_dir = '/content/CNN_dataset/CNN_dataset'

# Define the parameters for the ImageDataGenerator
batch_size = 32
img_height = 64
img_width = 64

# Create a list of all the image file paths and their corresponding labels
file_paths = []
labels = []
for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if not os.path.isdir(label_dir): continue
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        file_paths.append(img_path)
        labels.append(label)

# Split the dataset into training and test sets
train_file_paths, test_file_paths, train_labels, test_labels = train_test_split(
    file_paths,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Split the training set into training and validation sets
train_file_paths, val_file_paths, train_labels, val_labels = train_test_split(
    train_file_paths,
    train_labels,
    test_size=0.3,
    random_state=42,
    stratify=train_labels
)

# Create directories for the training, validation, and test sets
train_dir = '/content/train'
val_dir = '/content/val'
test_dir = '/content/test'
for dir_name in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
# Copy the training/validation/testing images to the training directory
for paths, labels, dir in [(train_file_paths, train_labels, train_dir), (val_file_paths, val_labels, val_dir), (test_file_paths, test_labels, test_dir)]:
    for file_path, label in zip(paths, labels):
        img_file = os.path.basename(file_path)
        label_dir = os.path.join(dir, label)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)
        shutil.copyfile(file_path, os.path.join(label_dir, img_file))

In [3]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the dataset path
dataset_path = "path/to/dataset"

# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

# Split the dataset into training, validation, and testing sets
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/val',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')


Found 30000 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.


In [4]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))


In [5]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=2,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size)

Epoch 1/2
937/937 [==============================] - 396s 410ms/step - loss: 0.1573 - accuracy: 0.9419 - val_loss: 0.0289 - val_accuracy: 0.9902
Epoch 2/2
937/937 [==============================] - 385s 411ms/step - loss: 0.0329 - accuracy: 0.9891 - val_loss: 0.0053 - val_accuracy: 0.9976


In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('Test accuracy:', test_acc)

313/313 - 13s - loss: 0.0067 - accuracy: 0.9971 - 13s/epoch - 43ms/step
Test accuracy: 0.9970999956130981
